In [1]:
import gym
import gym_banana

In [9]:
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical


#parser = argparse.ArgumentParser(description='PyTorch REINFORCE example')
# parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
#                     help='discount factor (default: 0.99)')
# parser.add_argument('--seed', type=int, default=543, metavar='N',
#                     help='random seed (default: 543)')
# parser.add_argument('--render', action='store_true',
#                     help='render the environment')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='interval between training status logs (default: 10)')
#args = parser.parse_args()


env = gym.make('Banana-v0')
env.seed(1)
torch.manual_seed(1)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(p=0.6)
#         self.affine2 = nn.Linear(128, 1)
        self.mu = nn.Linear(1,1, bias = True)
        self.tanh = nn.Tanh()

        self.saved_log_probs = []
        self.rewards = []
        self.actions = []

    def forward(self, x, y):
        x = self.affine1(x)
        x = self.dropout(x)
        mu_sigma = self.mu(y)
        action = self.tanh(x + mu_sigma)
#         action_scores = self.affine2(x)
#         return F.softmax(action_scores, dim=1)
        return action


policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state, previous_action):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state, previous_action)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()


def finish_episode():
    R = 0
    policy_loss = []
    returns = []
    for r in policy.rewards[::-1]:
        R = r + args.gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    for log_prob, R in zip(policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]


def main():
    running_reward = 10

    for i_episode in count(100):
        previous_action = 0
        state, ep_reward = env.reset(), 0
        for t in range(1, 10000):  # Don't infinite loop while learning
            action = select_action(state, previous_action)
            # the current price is 
#             state, reward, done, _ = env.step(action) #change banana
            if args.render:
                env.render()
#             policy.rewards.append(reward)
#             ep_reward += reward
            policy.actions.append(action)
            previous_action = action
            if done:
                break

#         running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode()
        if i_episode % args.log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break


if __name__ == '__main__':
    main()

2019-04-19 23:21:04,829 - root - INFO - BananaEnv - Version 0.1.0


TypeError: expected np.ndarray (got list)

[3]